# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [168]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [169]:
import tweepy
import math
import os.path
import pandas as pd
import json
import nltk


___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@YMartins123***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [170]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [171]:
#Produto escolhido:
produto = 'Samsung'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [172]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 429

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [210]:
#Limpando as mensagens 

import string
import random
data=pd.read_excel("Samsung.xlsx")
datatest=pd.read_excel("Samsung.xlsx",sheetname="Teste")

def sempontuacao(text):  #função que retira todos os sinais(emocotions continuam)
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

data["Treinamento"] = data['Treinamento'].apply(sempontuacao)
datatest["Teste"] = datatest["Teste"].apply(sempontuacao)
data.Treinamento=data.Treinamento.str.replace("  "," ")#retirando espaços vazios
datatest["Teste"] = datatest.Teste.str.replace("  "," ")


lista_rel = []
lista_nrel = []
prob_rel = 0
prob_nrel = 0


for i in range(len(data["Treinamento"])):
    if data["Relevancia"][i]=="Relevante":
        lista_rel+=(list(set(data["Treinamento"][i].split())))
    else:
        lista_nrel+=(list(set(data["Treinamento"][i].split())))
        
        
#lista_rel=  list(set(lista_rel))
#lista_nrel = list(set(lista_nrel))

n_rel = (data.Relevancia=="Relevante").sum()
n_nrel= (data.Relevancia=="Não Relevante").sum()

prob_rel=n_rel/data["Relevancia"].count()
prob_nrel=n_nrel/data["Relevancia"].count()

print(prob_rel)
print(prob_nrel)

#print((data.Relevancia=="Relevante").sum())


print(n_rel)
print(n_nrel)
print(prob_rel+prob_nrel)
print(data["Relevancia"].count())
print()
print()

result_prob=[]

for i in range(len(datatest.Teste)):
    result_prob.append([]) 
    for j in range(len(datatest.Teste[i].split())):
        count=lista_rel.count(datatest.Teste[i].split()[j])
       
        result_prob[i].append(count/n_rel)
        
        
        
        print("PALAVRA:{0} FRASE:{1}".format(datatest.Teste[i].split()[j],i))
        print(result_prob[i][j])

0.0666666666667
0.933333333333
20
280
1.0
300


PALAVRA:adoro FRASE:0
0.1
PALAVRA:samsung FRASE:0
1.0
PALAVRA:por FRASE:0
0.05
PALAVRA:causa FRASE:0
0.0
PALAVRA:dos FRASE:0
0.0
PALAVRA:temas FRASE:0
0.0
PALAVRA:fofinhos FRASE:0
0.0
PALAVRA:dia FRASE:1
0.0
PALAVRA:11 FRASE:1
0.0
PALAVRA:vcs FRASE:1
0.0
PALAVRA:pode FRASE:1
0.0
PALAVRA:me FRASE:1
0.1
PALAVRA:dar FRASE:1
0.0
PALAVRA:de FRASE:1
0.3
PALAVRA:presente FRASE:1
0.0
PALAVRA:um FRASE:1
0.1
PALAVRA:samsung FRASE:1
1.0
PALAVRA:s8 FRASE:1
0.1
PALAVRA:tá FRASE:1
0.05
PALAVRA:caso FRASE:2
0.0
PALAVRA:baseus FRASE:2
0.0
PALAVRA:original FRASE:2
0.0
PALAVRA:para FRASE:2
0.05
PALAVRA:samsung FRASE:2
1.0
PALAVRA:galaxy FRASE:2
0.05
PALAVRA:s7 FRASE:2
0.05
PALAVRA:estojo FRASE:2
0.0
PALAVRA:de FRASE:2
0.3
PALAVRA:luxo FRASE:2
0.0
PALAVRA:cristal FRASE:2
0.0
PALAVRA:tpu FRASE:2
0.0
PALAVRA:caso FRASE:2
0.0
PALAVRA:c FRASE:2
0.0
PALAVRA:https FRASE:2
0.15
PALAVRA:t FRASE:2
0.15
PALAVRA:co FRASE:2
0.15
PALAVRA:gupoo3m9gj FRASE:2
0.0
PALAVRA:e

0.0
PALAVRA:ter FRASE:37
0.05
PALAVRA:um FRASE:37
0.1
PALAVRA:samsung FRASE:37
1.0
PALAVRA:c FRASE:37
0.0
PALAVRA:aqls FRASE:37
0.0
PALAVRA:emojis FRASE:37
0.0
PALAVRA:e FRASE:37
0.2
PALAVRA:aql FRASE:37
0.0
PALAVRA:camera FRASE:37
0.0
PALAVRA:e FRASE:37
0.2
PALAVRA:era FRASE:37
0.0
PALAVRA:feliz FRASE:37
0.0
PALAVRA:serenidade FRASE:38
0.0
PALAVRA:no FRASE:38
0.05
PALAVRA:olhar FRASE:38
0.0
PALAVRA:de FRASE:38
0.3
PALAVRA:quem FRASE:38
0.05
PALAVRA:venseu FRASE:38
0.0
PALAVRA:quase FRASE:38
0.0
PALAVRA:tudo FRASE:38
0.05
PALAVRA:no FRASE:38
0.05
PALAVRA:fds FRASE:38
0.0
PALAVRA:e FRASE:38
0.2
PALAVRA:uma FRASE:38
0.05
PALAVRA:selfie FRASE:38
0.0
PALAVRA:com FRASE:38
0.15
PALAVRA:câmera FRASE:38
0.0
PALAVRA:frontal FRASE:38
0.0
PALAVRA:samsung FRASE:38
1.0
PALAVRA:https FRASE:38
0.15
PALAVRA:t FRASE:38
0.15
PALAVRA:co FRASE:38
0.15
PALAVRA:mlafb7ujzs FRASE:38
0.0
PALAVRA:a FRASE:39
0.25
PALAVRA:atualização FRASE:39
0.0
PALAVRA:da FRASE:39
0.6
PALAVRA:samsung FRASE:39
1.0
PALAVRA:ta FRA

PALAVRA:carregador FRASE:74
0.0
PALAVRA:de FRASE:74
0.3
PALAVRA:viagem FRASE:74
0.0
PALAVRA:samsung FRASE:74
1.0
PALAVRA:ultra FRASE:74
0.0
PALAVRA:rápido FRASE:74
0.0
PALAVRA:branco FRASE:74
0.0
PALAVRA:https FRASE:74
0.15
PALAVRA:t FRASE:74
0.15
PALAVRA:co FRASE:74
0.15
PALAVRA:qo… FRASE:74
0.0
PALAVRA:vendo FRASE:75
0.0
PALAVRA:notebook FRASE:75
0.0
PALAVRA:samsung FRASE:75
1.0
PALAVRA:i5 FRASE:75
0.0
PALAVRA:6gb FRASE:75
0.0
PALAVRA:ram FRASE:75
0.0
PALAVRA:1t FRASE:75
0.0
PALAVRA:hd FRASE:75
0.0
PALAVRA:1g FRASE:75
0.0
PALAVRA:placa FRASE:75
0.0
PALAVRA:de FRASE:75
0.3
PALAVRA:vídeo FRASE:75
0.0
PALAVRA:monitor FRASE:75
0.0
PALAVRA:15 FRASE:75
0.0
PALAVRA:aceito FRASE:75
0.0
PALAVRA:cartão FRASE:75
0.0
PALAVRA:de FRASE:75
0.3
PALAVRA:visita FRASE:75
0.0
PALAVRA:para FRASE:75
0.05
PALAVRA:https FRASE:75
0.15
PALAVRA:t FRASE:75
0.15
PALAVRA:co FRASE:75
0.15
PALAVRA:fiwxmys9l1 FRASE:75
0.0
PALAVRA:seu FRASE:76
0.0
PALAVRA:samsung FRASE:76
1.0
PALAVRA:vai FRASE:76
0.05
PALAVRA:receber

0.0
PALAVRA:eu FRASE:109
0.15
PALAVRA:bb FRASE:109
0.0
PALAVRA:frases FRASE:109
0.0
PALAVRA:de FRASE:109
0.3
PALAVRA:karol FRASE:109
0.0
PALAVRA:👏👏😂 FRASE:109
0.0
PALAVRA:gostei FRASE:110
0.0
PALAVRA:de FRASE:110
0.3
PALAVRA:um FRASE:110
0.1
PALAVRA:vídeo FRASE:110
0.0
PALAVRA:youtube FRASE:110
0.0
PALAVRA:https FRASE:110
0.15
PALAVRA:t FRASE:110
0.15
PALAVRA:co FRASE:110
0.15
PALAVRA:3ajyuzpnooão FRASE:110
0.0
PALAVRA:compre FRASE:110
0.0
PALAVRA:samsung FRASE:110
1.0
PALAVRA:s3 FRASE:110
0.0
PALAVRA:s4 FRASE:110
0.0
PALAVRA:s5 FRASE:110
0.0
PALAVRA:s6 FRASE:110
0.0
PALAVRA:s7 FRASE:110
0.05
PALAVRA:e FRASE:110
0.2
PALAVRA:s7 FRASE:110
0.05
PALAVRA:edge FRASE:110
0.0
PALAVRA:saiba FRASE:110
0.0
PALAVRA:porque FRASE:110
0.0
PALAVRA:adicionei FRASE:111
0.0
PALAVRA:um FRASE:111
0.1
PALAVRA:vídeo FRASE:111
0.0
PALAVRA:a FRASE:111
0.25
PALAVRA:uma FRASE:111
0.05
PALAVRA:playlist FRASE:111
0.0
PALAVRA:youtube FRASE:111
0.0
PALAVRA:https FRASE:111
0.15
PALAVRA:t FRASE:111
0.15
PALAVRA:co FRA

PALAVRA:um FRASE:151
0.1
PALAVRA:samsung FRASE:151
1.0
PALAVRA:no FRASE:151
0.05
PALAVRA:maximo FRASE:151
0.0
PALAVRA:https FRASE:151
0.15
PALAVRA:t FRASE:151
0.15
PALAVRA:co FRASE:151
0.15
PALAVRA:px1uxjdmja FRASE:151
0.0
PALAVRA:pesquisem FRASE:152
0.0
PALAVRA:samsung FRASE:152
1.0
PALAVRA:q FRASE:152
0.0
PALAVRA:led FRASE:152
0.0
PALAVRA:é FRASE:152
0.45
PALAVRA:uma FRASE:152
0.05
PALAVRA:das FRASE:152
0.0
PALAVRA:televisões FRASE:152
0.05
PALAVRA:mais FRASE:152
0.05
PALAVRA:fodas FRASE:152
0.0
PALAVRA:que FRASE:152
0.35
PALAVRA:você FRASE:152
0.0
PALAVRA:possa FRASE:152
0.0
PALAVRA:imaginar FRASE:152
0.0
PALAVRA:definitivamente FRASE:153
0.0
PALAVRA:snap FRASE:153
0.0
PALAVRA:não FRASE:153
0.1
PALAVRA:foi FRASE:153
0.0
PALAVRA:feito FRASE:153
0.0
PALAVRA:pra FRASE:153
0.05
PALAVRA:samsung FRASE:153
1.0
PALAVRA:meu FRASE:154
0.05
PALAVRA:pai FRASE:154
0.0
PALAVRA:esta FRASE:154
0.0
PALAVRA:vendendo FRASE:154
0.0
PALAVRA:uma FRASE:154
0.05
PALAVRA:tv FRASE:154
0.05
PALAVRA:samsung FR

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
